# 第23章：综合项目 - 智能问答系统

## 项目概述

本章将构建一个**完整的企业文档智能问答系统**，整合前面学习的所有知识：

### 🎯 项目目标

构建一个基于RAG（Retrieval Augmented Generation）的问答系统，能够：
- 📄 加载和处理文档
- 🔍 智能检索相关内容
- 💬 生成准确的答案
- 📝 引用信息来源
- ⚡ 支持流式输出
- 📊 评估系统性能

### 🛠 技术栈

| 组件 | 技术 | 作用 |
|------|------|------|
| **文档加载** | Document Loaders | 读取各种格式文档 |
| **文本分割** | Text Splitters | 将文档切分成chunks |
| **向量化** | Embeddings | 将文本转换为向量 |
| **向量存储** | Vector Store | 存储和检索向量 |
| **检索** | Retriever | 查找相关文档 |
| **生成** | RAG Agent | 基于检索内容生成答案 |
| **评估** | Evaluators | 评估答案质量 |

### 📋 系统架构

```
文档输入
  ↓
文档加载器 (Document Loader)
  ↓
文本分割器 (Text Splitter)
  ↓
向量化 (Embeddings)
  ↓
向量存储 (Vector Store)
  ↓
用户提问 → 检索器 (Retriever) → RAG Agent → 答案输出
                                      ↓
                                   引用来源
```

### 💡 核心特性

1. **多格式支持** - 支持TXT、Markdown等文本格式
2. **智能检索** - 基于语义相似度检索
3. **引用来源** - 答案附带文档来源
4. **流式输出** - 实时显示生成过程
5. **质量评估** - 自动评估答案质量

---

## 学习收获

通过本项目，你将学会：
- ✅ 如何构建完整的RAG应用
- ✅ 如何选择和配置各个组件
- ✅ 如何优化检索和生成质量
- ✅ 如何评估系统性能
- ✅ 如何处理实际场景中的问题


In [2]:
# 环境配置和依赖导入
import os
import sys
from importlib import reload

_project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(_project_root)

from config import config

# 导入LangChain核心组件
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.agents import create_agent
from langchain.tools import tool

print("✅ 环境配置完成！")
print(f"项目根目录: {_project_root}")
print("\n📦 已导入的核心组件：")
print("  - ChatOpenAI: 大语言模型")
print("  - OpenAIEmbeddings: 文本向量化")
print("  - TextLoader: 文档加载器")
print("  - RecursiveCharacterTextSplitter: 文本分割器")
print("  - InMemoryVectorStore: 向量存储")
print("  - create_agent: Agent创建")
print("\n🚀 准备构建智能问答系统！")

✅ 环境配置完成！
项目根目录: /home/iip/tp/038-project/rag

📦 已导入的核心组件：
  - ChatOpenAI: 大语言模型
  - OpenAIEmbeddings: 文本向量化
  - TextLoader: 文档加载器
  - RecursiveCharacterTextSplitter: 文本分割器
  - InMemoryVectorStore: 向量存储
  - create_agent: Agent创建

🚀 准备构建智能问答系统！


## 步骤1：文档加载与处理

加载文档并分割成适合检索的chunks。

In [3]:
print("【步骤1：加载和分割文档】\n")

# 1. 加载文档
print("=== 加载文档 ===\n")
doc_path = os.path.join(_project_root, "docs", "sample_doc.txt")

loader = TextLoader(doc_path, encoding='utf-8')
documents = loader.load()

print(f"✓ 加载文档: {doc_path}")
print(f"  文档数量: {len(documents)}")
print(f"  文档长度: {len(documents[0].page_content)} 字符")
print(f"  文档预览: {documents[0].page_content[:100]}...")
print()

# 2. 分割文档
print("=== 分割文档 ===\n")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,      # 每个chunk最多500字符
    chunk_overlap=50,    # chunk之间重叠50字符
    length_function=len,
    separators=["\n\n", "\n", "。", "！", "？", " ", ""]
)

doc_splits = text_splitter.split_documents(documents)

print(f"✓ 文档分割完成")
print(f"  分割后的chunks数量: {len(doc_splits)}")
print(f"\n前3个chunks预览：")
for i, chunk in enumerate(doc_splits[:3], 1):
    print(f"\n  Chunk {i} ({len(chunk.page_content)} 字符):")
    print(f"  {chunk.page_content[:80]}...")

print(f"\n💡 分割策略：")
print(f"  - chunk_size=500: 平衡上下文和检索精度")
print(f"  - chunk_overlap=50: 保持语义连贯性")
print(f"  - 按段落、句子分割: 保持语义完整")


【步骤1：加载和分割文档】

=== 加载文档 ===

✓ 加载文档: /home/iip/tp/038-project/rag/docs/sample_doc.txt
  文档数量: 1
  文档长度: 724 字符
  文档预览: # LangChain 学习指南

## 什么是LangChain？

LangChain是一个强大的开源框架，专门用于构建基于大语言模型（LLM）的应用程序。它提供了一套完整的工具和抽象，使开发者能...

=== 分割文档 ===

✓ 文档分割完成
  分割后的chunks数量: 2

前3个chunks预览：

  Chunk 1 (460 字符):
  # LangChain 学习指南

## 什么是LangChain？

LangChain是一个强大的开源框架，专门用于构建基于大语言模型（LLM）的应用程序。...

  Chunk 2 (302 字符):
  检索增强生成（RAG）是LangChain的核心应用之一。它通过以下步骤工作：

1. 文档加载：使用Document Loaders读取各种格式的文档
2. ...

💡 分割策略：
  - chunk_size=500: 平衡上下文和检索精度
  - chunk_overlap=50: 保持语义连贯性
  - 按段落、句子分割: 保持语义完整


## 步骤2：向量化与存储

将文档chunks转换为向量并存储，支持语义检索。

In [4]:
print("【步骤2：向量化与存储】\n")

# 初始化Embeddings模型
print("=== 初始化Embeddings模型 ===\n")
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    api_key=config.CLOUD_API_KEY,
    base_url=config.CLOUD_BASE_URL
)
print(f"✓ Embeddings模型: {embeddings.model}")
print()

# 创建向量存储并索引文档
print("=== 创建向量存储 ===\n")
vectorstore = InMemoryVectorStore.from_documents(
    documents=doc_splits,
    embedding=embeddings
)
print(f"✓ 向量存储创建完成")
print(f"  已索引 {len(doc_splits)} 个文档chunks")
print()

# 测试检索功能
print("=== 测试检索功能 ===\n")
test_query = "什么是LangChain？"
print(f"测试查询: {test_query}\n")

# 检索最相关的3个chunks
retrieved_docs = vectorstore.similarity_search(test_query, k=3)

print(f"检索到 {len(retrieved_docs)} 个相关chunks：\n")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"Chunk {i}:")
    print(f"  内容: {doc.page_content[:100]}...")
    print(f"  长度: {len(doc.page_content)} 字符\n")

print("💡 检索工作原理：")
print("  1. 将查询转换为向量")
print("  2. 计算与所有chunks的相似度")
print("  3. 返回最相似的top-k个chunks")
print("  4. 这些chunks将用于生成答案")

【步骤2：向量化与存储】

=== 初始化Embeddings模型 ===

✓ Embeddings模型: text-embedding-3-small

=== 创建向量存储 ===

✓ 向量存储创建完成
  已索引 2 个文档chunks

=== 测试检索功能 ===

测试查询: 什么是LangChain？

检索到 2 个相关chunks：

Chunk 1:
  内容: # LangChain 学习指南

## 什么是LangChain？

LangChain是一个强大的开源框架，专门用于构建基于大语言模型（LLM）的应用程序。它提供了一套完整的工具和抽象，使开发者能...
  长度: 460 字符

Chunk 2:
  内容: 检索增强生成（RAG）是LangChain的核心应用之一。它通过以下步骤工作：

1. 文档加载：使用Document Loaders读取各种格式的文档
2. 文本分割：将长文档切分成小块
3. 向量...
  长度: 302 字符

💡 检索工作原理：
  1. 将查询转换为向量
  2. 计算与所有chunks的相似度
  3. 返回最相似的top-k个chunks
  4. 这些chunks将用于生成答案


## 步骤3：构建RAG Agent

创建一个智能Agent，能够检索文档并生成答案。

In [5]:
print("【步骤3：构建RAG Agent】\n")

# 1. 创建检索工具
print("=== 创建检索工具 ===\n")

@tool(response_format="content_and_artifact")
def retrieve_documents(query: str):
    """检索与查询相关的文档内容"""
    # 检索top-3相关文档
    docs = vectorstore.similarity_search(query, k=3)
    
    # 格式化文档内容
    formatted_docs = "\n\n".join([
        f"【文档片段 {i+1}】\n{doc.page_content}"
        for i, doc in enumerate(docs)
    ])
    
    # 返回格式化内容和原始文档（artifact）
    return formatted_docs, docs

print("✓ 创建检索工具: retrieve_documents")
print("  - 检索top-3相关文档")
print("  - 返回格式化内容和原始文档")
print()

# 2. 初始化模型
print("=== 初始化LLM ===\n")
llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    api_key=config.CLOUD_API_KEY,
    base_url=config.CLOUD_BASE_URL,
)
print(f"✓ 模型: {llm.model_name}")
print()

# 3. 创建RAG Agent
print("=== 创建RAG Agent ===\n")
rag_agent = create_agent(
    model=llm,
    tools=[retrieve_documents],
    system_prompt="""你是一个专业的文档问答助手。

工作流程：
1. 使用retrieve_documents工具检索相关文档
2. 仔细阅读检索到的文档内容
3. 基于文档内容回答用户问题
4. 如果文档中没有相关信息，明确告知用户

要求：
- 答案必须基于检索到的文档内容
- 保持简洁，3-5句话
- 如果有多个相关信息，综合回答
- 不要编造文档中没有的信息"""
)

print("✓ RAG Agent创建完成")
print("  - 工具: retrieve_documents")
print("  - 系统提示: 基于文档回答问题")
print("\n🎉 RAG系统构建完成！")

【步骤3：构建RAG Agent】

=== 创建检索工具 ===

✓ 创建检索工具: retrieve_documents
  - 检索top-3相关文档
  - 返回格式化内容和原始文档

=== 初始化LLM ===

✓ 模型: gpt-4.1-mini

=== 创建RAG Agent ===

✓ RAG Agent创建完成
  - 工具: retrieve_documents
  - 系统提示: 基于文档回答问题

🎉 RAG系统构建完成！


## 步骤4：测试RAG系统

测试系统的问答能力和引用功能。

In [6]:
print("【步骤4：测试RAG系统】\n")

# 定义测试问题
test_questions = [
    "什么是LangChain？",
    "LangChain有哪些核心组件？",
    "RAG技术是如何工作的？",
    "LangChain适合什么使用场景？"
]

print("=== 测试问答功能 ===\n")

for i, question in enumerate(test_questions, 1):
    print(f"【问题 {i}】{question}\n")
    
    # 调用RAG Agent
    result = rag_agent.invoke({
        "messages": [{"role": "user", "content": question}]
    })
    
    # 获取答案
    answer = result['messages'][-1].content
    print(f"回答: {answer}\n")
    
    # 查看是否调用了检索工具
    tool_messages = [msg for msg in result['messages'] if hasattr(msg, 'type') and msg.type == 'tool']
    if tool_messages:
        print(f"✓ 使用了检索工具")
        # 获取检索到的文档（从artifact）
        for tool_msg in tool_messages:
            if hasattr(tool_msg, 'artifact') and tool_msg.artifact:
                print(f"  检索了 {len(tool_msg.artifact)} 个文档片段")
    
    print("-" * 70)
    print()

print("💡 观察要点：")
print("  ✓ Agent自动决定何时使用检索工具")
print("  ✓ 答案基于检索到的文档内容")
print("  ✓ 可以通过artifact获取原始文档")
print("  ✓ 所有执行过程都被LangSmith追踪")

【步骤4：测试RAG系统】

=== 测试问答功能 ===

【问题 1】什么是LangChain？

回答: LangChain是一个开源框架，专门用于构建基于大语言模型（LLM）的应用程序。它提供了模型、提示词、链、代理和记忆等核心组件，方便开发者集成和管理LLM。LangChain的核心应用之一是检索增强生成（RAG），通过文档加载、文本分割、向量化、存储、检索和生成步骤，实现基于文档的智能问答和内容生成。常见应用包括智能客服、文档问答、代码助手和内容生成工具。

✓ 使用了检索工具
  检索了 2 个文档片段
----------------------------------------------------------------------

【问题 2】LangChain有哪些核心组件？

回答: LangChain的核心组件包括：1. Models（模型），支持多种大语言模型提供商；2. Prompts（提示词），提供提示词模板和管理工具；3. Chains（链），用于将多个组件串联形成复杂流程；4. Agents（代理），根据输入动态调用工具实现智能交互；5. Memory（记忆），为对话系统提供上下文记忆能力。

✓ 使用了检索工具
  检索了 2 个文档片段
----------------------------------------------------------------------

【问题 3】RAG技术是如何工作的？

回答: RAG（检索增强生成）技术的工作流程包括：首先使用文档加载器读取各种格式的文档，然后将长文档切分成小块，接着将文本转换为向量表示并存储在向量数据库中。用户提出问题时，系统会检索相关文档内容，最后基于检索到的内容生成答案。该技术广泛应用于智能客服、文档问答、代码助手等场景。

✓ 使用了检索工具
  检索了 2 个文档片段
----------------------------------------------------------------------

【问题 4】LangChain适合什么使用场景？

回答: LangChain适合用于构建基于大语言模型的各种应用，典型使用场景包括智能客服系统、文档问答系统、代码助手、内容生成工具和数据分析助手。它通过检索增强生成（RAG）技术，实现文

## 步骤5：流式输出

实现实时流式输出，提升用户体验。

In [7]:
print("【步骤5：流式输出】\n")

import time

question = "请详细介绍LangChain的RAG技术"
print(f"问题: {question}\n")
print("回答（流式）: ", end="", flush=True)

# 使用stream模式
for chunk in rag_agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="messages"
):
    # 只输出AI的内容
    if isinstance(chunk, tuple) and len(chunk) == 2:
        message, metadata = chunk
        if hasattr(message, 'content') and message.content:
            print(message.content, end="", flush=True)
            time.sleep(0.02)  # 模拟打字效果

print("\n\n✓ 流式输出完成")
print("\n💡 流式输出的优势：")
print("  - 用户可以立即看到响应")
print("  - 提升用户体验")
print("  - 适合长文本生成")
print("  - 可以提前中断不需要的输出")

【步骤5：流式输出】

问题: 请详细介绍LangChain的RAG技术

回答（流式）: 【文档片段 1】
# LangChain 学习指南

## 什么是LangChain？

LangChain是一个强大的开源框架，专门用于构建基于大语言模型（LLM）的应用程序。它提供了一套完整的工具和抽象，使开发者能够轻松地将LLM集成到各种应用场景中。

## 核心组件

### 1. Models（模型）
LangChain支持多种LLM提供商，包括OpenAI、Anthropic、Google等。通过统一的接口，开发者可以轻松切换不同的模型。

### 2. Prompts（提示词）
提供了Prompt模板和管理工具，帮助开发者更好地设计和优化提示词。

### 3. Chains（链）
将多个组件串联起来，形成复杂的处理流程。

### 4. Agents（代理）
Agent可以根据用户输入动态决定调用哪些工具，实现更智能的交互。

### 5. Memory（记忆）
为对话系统提供上下文记忆能力，使对话更加连贯。

## RAG技术

检索增强生成（RAG）是LangChain的核心应用之一。它通过以下步骤工作：

【文档片段 2】
检索增强生成（RAG）是LangChain的核心应用之一。它通过以下步骤工作：

1. 文档加载：使用Document Loaders读取各种格式的文档
2. 文本分割：将长文档切分成小块
3. 向量化：将文本转换为向量表示
4. 存储：将向量存储在向量数据库中
5. 检索：根据用户问题检索相关文档
6. 生成：基于检索到的内容生成答案

## 使用场景

- 智能客服系统
- 文档问答系统
- 代码助手
- 内容生成工具
- 数据分析助手

## 最佳实践

1. 合理设计Prompt模板
2. 选择合适的文本分割策略
3. 优化向量检索参数
4. 使用评估工具持续改进
5. 注意成本和性能平衡LangChain的RAG（检索增强生成）技术是一种结合文档检索与生成模型的技术。其工作流程包括：首先使用Document Loaders加载各种格式的文档，然后将长文档切分成小块，接着将文本向量化并存储到向量数据库中。用户提问时，系统会检索相关文档，再基于检索结果生成答案。RAG技术广泛应用于智能客服、文档问答、代码助手等场景，且在使用时需

## 步骤6：系统评估

评估RAG系统的性能和质量。

In [8]:
print("【步骤6：系统评估】\n")

# 创建评估数据集
eval_dataset = [
    {
        "question": "什么是LangChain？",
        "expected_keywords": ["框架", "LLM", "应用"]
    },
    {
        "question": "LangChain的核心组件有哪些？",
        "expected_keywords": ["Models", "Prompts", "Chains", "Agents"]
    },
    {
        "question": "RAG技术包含哪些步骤？",
        "expected_keywords": ["文档加载", "分割", "向量化", "检索", "生成"]
    }
]

print("=== 运行评估 ===\n")

# 定义评估器
def keyword_coverage_evaluator(question, answer, expected_keywords):
    """评估答案是否包含关键词"""
    answer_lower = answer.lower()
    found = [kw for kw in expected_keywords if kw.lower() in answer_lower]
    score = len(found) / len(expected_keywords)
    return score, found

# 评估每个问题
results = []
for i, test_case in enumerate(eval_dataset, 1):
    print(f"评估样本 {i}/{len(eval_dataset)}: {test_case['question'][:30]}...")
    
    # 运行RAG系统
    result = rag_agent.invoke({
        "messages": [{"role": "user", "content": test_case['question']}]
    })
    answer = result['messages'][-1].content
    
    # 评估
    score, found_keywords = keyword_coverage_evaluator(
        test_case['question'],
        answer,
        test_case['expected_keywords']
    )
    
    results.append({
        "question": test_case['question'],
        "answer": answer[:80] + "...",
        "score": score,
        "found_keywords": found_keywords
    })
    
    print(f"  关键词覆盖率: {score:.2%}")
    print(f"  找到关键词: {found_keywords}")
    print()

# 汇总结果
avg_score = sum(r['score'] for r in results) / len(results)

print("=== 评估汇总 ===\n")
print(f"平均关键词覆盖率: {avg_score:.2%}")
print(f"测试样本数: {len(results)}")
print()

print("💡 评估要点：")
print("  ✓ 多维度评估：关键词、相关性、忠实度")
print("  ✓ 使用测试集持续监控质量")
print("  ✓ 结合LangSmith进行在线评估")
print("  ✓ 根据评估结果优化系统")

【步骤6：系统评估】

=== 运行评估 ===

评估样本 1/3: 什么是LangChain？...
  关键词覆盖率: 100.00%
  找到关键词: ['框架', 'LLM', '应用']

评估样本 2/3: LangChain的核心组件有哪些？...
  关键词覆盖率: 100.00%
  找到关键词: ['Models', 'Prompts', 'Chains', 'Agents']

评估样本 3/3: RAG技术包含哪些步骤？...
  关键词覆盖率: 100.00%
  找到关键词: ['文档加载', '分割', '向量化', '检索', '生成']

=== 评估汇总 ===

平均关键词覆盖率: 100.00%
测试样本数: 3

💡 评估要点：
  ✓ 多维度评估：关键词、相关性、忠实度
  ✓ 使用测试集持续监控质量
  ✓ 结合LangSmith进行在线评估
  ✓ 根据评估结果优化系统


## 项目总结

### 🎉 完成的功能

本项目实现了一个完整的RAG智能问答系统：

| 功能模块 | 实现方式 | 关键技术 |
|---------|---------|---------|
| **文档处理** | TextLoader + TextSplitter | 加载和分割文档 |
| **向量检索** | OpenAIEmbeddings + InMemoryVectorStore | 语义检索 |
| **智能问答** | RAG Agent + retrieve_documents工具 | 动态检索和生成 |
| **引用来源** | response_format="content_and_artifact" | 返回原始文档 |
| **流式输出** | stream_mode="messages" | 实时响应 |
| **质量评估** | 关键词覆盖率评估器 | 自动化评估 |

### 🔑 关键技术点

#### 1. 文档处理流程

```python
文档 → TextLoader → 加载
     → RecursiveCharacterTextSplitter → 分割
     → OpenAIEmbeddings → 向量化
     → InMemoryVectorStore → 存储
```

#### 2. RAG Agent设计

```python
@tool(response_format="content_and_artifact")
def retrieve_documents(query: str):
    docs = vectorstore.similarity_search(query, k=3)
    return formatted_content, docs  # 内容 + 原始文档

agent = create_agent(
    model=llm,
    tools=[retrieve_documents],
    system_prompt="基于文档回答问题..."
)
```

#### 3. 检索参数优化

| 参数 | 作用 | 推荐值 |
|------|------|--------|
| `chunk_size` | chunk大小 | 500-1000 |
| `chunk_overlap` | 重叠大小 | 50-100 |
| `k` | 检索数量 | 3-5 |
| `temperature` | 生成随机性 | 0-0.3（RAG场景） |

### 💡 优化建议

#### 性能优化

1. **检索优化**
   - 调整chunk_size平衡上下文和精度
   - 增加chunk_overlap保持语义连贯
   - 使用MMR（最大边际相关性）避免重复

2. **生成优化**
   - 设置temperature=0提高确定性
   - 优化system_prompt引导答案格式
   - 限制答案长度避免冗余

3. **成本优化**
   - 使用较小的embedding模型
   - 减少检索的k值
   - 使用缓存避免重复计算

#### 质量优化

1. **提高准确性**
   - 优化文档分割策略
   - 增加检索的文档数量
   - 使用更强的LLM模型

2. **提高相关性**
   - 改进检索算法（混合检索）
   - 过滤低质量文档
   - 重排序检索结果

3. **提高忠实度**
   - 强调"基于文档回答"的提示
   - 添加引用来源
   - 使用评估器检测幻觉

### 🚀 扩展方向

1. **多文档支持** - 支持PDF、Word、网页等
2. **混合检索** - 结合关键词和语义检索
3. **多轮对话** - 添加Checkpointer支持上下文
4. **答案评分** - 返回置信度分数
5. **引用展示** - 在答案中标注来源
6. **用户反馈** - 收集反馈持续优化

### 📊 性能指标

评估RAG系统的关键指标：

| 指标 | 说明 | 目标值 |
|------|------|--------|
| **检索准确率** | 检索到的文档是否相关 | > 80% |
| **答案相关性** | 答案是否回答了问题 | > 90% |
| **答案忠实度** | 答案是否基于文档 | > 95% |
| **响应延迟** | 从问题到答案的时间 | < 3秒 |
| **Token成本** | 每次查询的token使用 | 优化 |

---

## 关键收获

✅ **完整的RAG流程** - 从文档到答案的端到端实现

✅ **Agent vs Chain** - Agent更灵活，可以动态决定是否检索

✅ **response_format="content_and_artifact"** - 同时返回内容和原始文档

✅ **流式输出** - 提升用户体验的关键

✅ **评估驱动优化** - 基于数据持续改进

✅ **LangSmith集成** - 所有操作自动追踪，便于调试

✅ **模块化设计** - 每个组件可独立替换和优化

---

## 下一步

完成智能问答系统后，可以：

1. **优化系统** - 尝试不同的分割策略、检索参数
2. **添加功能** - 多轮对话、引用展示、答案评分
3. **第24章** - 构建对话机器人项目
4. **部署上线** - 将系统部署到生产环境

🎉 恭喜！你已经完成了第一个完整的RAG项目！
